In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import pandas as pd

Using TensorFlow backend.
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
                names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
                # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [3]:
dataset = read_csv('dummy.csv')
dataset= dataset.drop(columns=["newCol","ID"])
values = dataset.values
print(dataset.head(5))

validation = read_csv('dummy_validate.csv')
validation= validation.drop(columns=["newCol","ID"])
values_validation = validation.values

      Hours   Minutes   Seconds   Latency      Cost  Reliability    Friday  \
0 -0.214634 -1.641064 -0.570284  0.398253 -1.799170     0.064093 -0.400349   
1  1.092639 -1.007746  0.051433 -0.079609 -1.738793     0.064093 -0.400349   
2  0.075871 -1.180469  0.158475 -0.406889  0.817158     0.064093 -0.400349   
3 -0.069382  1.180082 -0.012426 -0.406794  0.314018     0.064093 -0.400349   
4 -0.940897  1.180082 -1.456998  0.341495  0.384458     0.064093 -0.400349   

     Monday  Saturday    Sunday  ...  Server_4.0  Server_5.0  Server_6.0  \
0 -0.435389 -0.392927 -0.402323  ...    -0.31139    2.577480   -0.388116   
1 -0.435389 -0.392927 -0.402323  ...    -0.31139    2.577480   -0.388116   
2 -0.435389 -0.392927 -0.402323  ...    -0.31139   -0.387967   -0.388116   
3 -0.435389 -0.392927 -0.402323  ...    -0.31139   -0.387967   -0.388116   
4 -0.435389 -0.392927 -0.402323  ...    -0.31139   -0.387967   -0.388116   

   Server_7.0  Server_8.0  Tactic_1.0  Tactic_2.0  Tactic_3.0  Tactic_4.0 

In [4]:
reframed = series_to_supervised(values, 1, 1)
reframed_validation = series_to_supervised(values_validation, 1, 1)

reframed.drop(reframed.columns[[26,27,28,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]], axis=1, inplace=True)
reframed_validation.drop(reframed_validation.columns[[26,27,28,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]], axis=1, inplace=True)
print(reframed.head(1))

   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)  var6(t-1)  \
1  -0.214634  -1.641064  -0.570284   0.398253   -1.79917   0.064093   

   var7(t-1)  var8(t-1)  var9(t-1)  var10(t-1)  ...  var20(t-1)  var21(t-1)  \
1  -0.400349  -0.435389  -0.392927   -0.402323  ...   -0.389043   -0.380283   

   var22(t-1)  var23(t-1)  var24(t-1)  var25(t-1)  var26(t-1)   var4(t)  \
1    1.972332   -0.498444   -0.497559    -0.49892   -0.498035 -0.079609   

    var5(t)   var6(t)  
1 -1.738793  0.064093  

[1 rows x 29 columns]


In [5]:
train = reframed.values
test = reframed_validation.values

train_X, train_y = train[:, :-3], train[:,-3:]
test_X, test_y = test[:, :-3], test[:,-3:]

train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(36761, 1, 26) (36761, 3) (9190, 1, 26) (9190, 3)


In [ ]:
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(100, return_sequences=True,input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(units=3))

model.compile(loss='mse', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=20, batch_size=72, validation_data=(test_X, test_y), verbose=1, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Train on 36761 samples, validate on 9190 samples
Epoch 1/20
36761/36761 [==============================] - 6s 173us/step - loss: 1.0056 - val_loss: 0.9781
Epoch 2/20
36761/36761 [==============================] - 3s 95us/step - loss: 1.0053 - val_loss: 0.9781
Epoch 3/20
36761/36761 [==============================] - 4s 100us/step - loss: 1.0050 - val_loss: 0.9784
Epoch 4/20
36761/36761 [==============================] - 4s 106us/step - loss: 1.0046 - val_loss: 0.9789
Epoch 5/20
36761/36761 [==============================] - 3s 93us/step - loss: 1.0040 - val_loss: 0.9796
Epoch 6/20
36761/36761 [==============================] - 4s 108us/step - loss: 1.0033 - val_loss: 0.9804
Epoch 7/20
36761/36761 [==============================] - 3s 84us/step - loss: 1.0025 - val_loss: 0.9813
Epoch 8/20
36761/36761 [==============================] - 3s 83us/step - loss: 1.0015 - val_loss: 0.9825
Epoch 9/20
36761/36761 [==============================] - 4s 100us/step - loss: 1.0004 - val_loss: 0.9838
E

In [ ]:
test_dataset = read_csv('dummy_1.csv')
test_dataset= test_dataset.drop(columns=["newCol","ID"])
test_values = test_dataset.values
reframed_test = series_to_supervised(test_values, 1, 1)
reframed_test.drop(reframed_test.columns[[26,27,28,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]], axis=1, inplace=True)
testset = reframed_test.values
testset_X, testset_y = testset[:, :-3], testset[:,-3:]
testdataReshaped = testset_X.reshape((testset_X.shape[0], 1, testset_X.shape[1]))

In [ ]:
import pandas as pd

yhat = model.predict(testdataReshaped)

print(yhat)
dataset = pd.DataFrame({'predicted_Latency': yhat[:, 0], 'predicted_Cost': yhat[:, 1],
                       'predicted_Reliability': yhat[:, 2]})
dataset['predicted_Reliability'].loc[dataset['predicted_Reliability'] >0.5] = 1
dataset['predicted_Reliability'].loc[dataset['predicted_Reliability'] <0.5] = 0

In [ ]:
frames = [test_dataset, dataset]
result = pd.concat(frames,axis =1)

In [ ]:
import numpy as np
result.head(10)
result.to_csv('predictions_LSTM.csv', sep=',', index=0)

In [ ]:
print(testset_y)